Nessie Spark Test Env with NBA Dataset
============================
This testcreates 2 branches and creates 2 tables within the dev branch .It uses Nessie Python API along with Spark3 from Iceberg.
Prereq : Need Nessie server running. (The location will be set in the script below) 
Need read/write access to an hdfs store (The location will be set below)

Initialize Pyspark
----------------------------------------------
To get started, we will first have to do a few setup steps that give us everything we need
to get started with Nessie. In case you're interested in the detailed setup steps for Spark, you can check out the [docs](https://projectnessie.org/tools/spark/).

The Binder server has downloaded spark and some data for us as well as started a Nessie server in the background. All we have to do is start Spark.

The below cell starts a local Spark session with parameters needed to configure Nessie. Each config option is followed by a comment explaining its purpose.


In [24]:
import os
import findspark
from pyspark.sql import *
from pyspark import SparkConf
import pynessie
findspark.init()
pynessie_version = "0.9.2"

conf = SparkConf()
# we need iceberg libraries and the nessie sql extensions
conf.set("spark.jars.packages", f"org.apache.iceberg:iceberg-spark3-runtime:0.12.0,org.projectnessie:nessie-spark-extensions:{pynessie_version}")
# ensure python <-> java interactions are w/ pyarrow
conf.set("spark.sql.execution.pyarrow.enabled", "true")
# create catalog dev_catalog as an iceberg catalog
conf.set("spark.sql.catalog.dev_catalog", "org.apache.iceberg.spark.SparkCatalog")
# tell the dev_catalog that its a Nessie catalog
conf.set("spark.sql.catalog.dev_catalog.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
# set the location for Nessie catalog to store data. Spark writes to this directory
#setup the iceberg backing store in the local file system
#conf.set("spark.sql.catalog.dev_catalog.warehouse", 'file://' + os.getcwd() + '/spark_warehouse/iceberg')
#setup the iceberg backing store in s3 (not working)
#conf.set("spark.sql.catalog.dev_catalog.warehouse", 's3a://tmp.dremio.com/sanfolder'/spark_warehouse/iceberg')
#setup the iceberg backing store in hdfs (need read/write access  to the location )
conf.set("spark.sql.catalog.dev_catalog.warehouse", 'hdfs://localhost:8020' + '/ddp'+'/spark_warehouse/iceberg')

# set the url location of the nessie server. In this example  its running locally. There are many ways to run it (see https://projectnessie.org/try/)
conf.set("spark.sql.catalog.dev_catalog.uri", "http://localhost:19120/api/v1")
# default branch for Nessie catalog to work on
conf.set("spark.sql.catalog.dev_catalog.ref", "main")
# use no authorization. Options are NONE AWS BASIC and aws implies running Nessie on a lambda
conf.set("spark.sql.catalog.dev_catalog.auth_type", "NONE")
# enable the extensions for both Nessie and Iceberg
conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
# finally, start up the Spark server
spark = SparkSession.builder.config(conf=conf).getOrCreate()

#S3 setup (not working )
#spark._jsc.hadoopConfiguration().set("fs.s3a.awsAccessKeyId", "")
#spark._jsc.hadoopConfiguration().set("fs.s3a.awsSecretAccessKey","" )
#spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#spark._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
#hadoop_conf = spark._jsc.hadoopConfiguration()

print("Spark Running")

Spark Running


Test description
============================

In this Demo we are a data engineer working at a fictional sports analytics blog. In order for the authors to write articles they have to have access to the relevant data. They need to be able to retrieve data quickly and be able to create charts with it.

We have been asked to collect and expose some information about basketball players. We have located some data sources and are now ready to start ingesting data into our data lakehouse. We will perform the ingestion steps on a Nessie branch to test and validate the data before exposing to the analysts.

Set up  branches
----------------------------
Once all dependencies are configured, we can get started with ingesting our basketball data into `Nessie` with the following steps:

- Create a new branch named `dev`
- List all branches

It is worth mentioning that we don't have to explicitly create a `main` branch, since it's the default branch.

In [25]:
spark.sql("DROP BRANCH dev IN dev_catalog")

Py4JJavaError: An error occurred while calling o42.sql.
: org.projectnessie.error.NessieNotFoundException: Unable to find reference [dev].
	at org.projectnessie.client.rest.ResponseCheckFilter.checkResponse(ResponseCheckFilter.java:61)
	at org.projectnessie.client.rest.NessieHttpResponseFilter.filter(NessieHttpResponseFilter.java:35)
	at org.projectnessie.client.http.HttpRequest.lambda$executeRequest$6(HttpRequest.java:139)
	at java.util.ArrayList.forEach(ArrayList.java:1257)
	at org.projectnessie.client.http.HttpRequest.executeRequest(HttpRequest.java:139)
	at org.projectnessie.client.http.HttpRequest.get(HttpRequest.java:170)
	at org.projectnessie.client.ClientTreeApi.getReferenceByName(ClientTreeApi.java:66)
	at sun.reflect.GeneratedMethodAccessor22.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at org.projectnessie.client.NessieHttpClient$ExceptionRewriter.invoke(

In [26]:
spark.sql("CREATE BRANCH dev IN dev_catalog AS main").toPandas()

,refType,name,hash
0,Branch,dev,da0ea1e71d020987d2f2686deb6c9e80266b7d1e338903...


We have created the branch `dev` and we can see the branch with the Nessie `hash` its currently pointing to. 

Below we list all branches. Note that the auto created `main` branch already exists and both branches point at the same `hash`

In [27]:
spark.sql("LIST REFERENCES IN dev_catalog").toPandas() 

,refType,name,hash
0,Branch,main,da0ea1e71d020987d2f2686deb6c9e80266b7d1e338903...
1,Branch,dev,da0ea1e71d020987d2f2686deb6c9e80266b7d1e338903...


Create tables under dev branch
-------------------------------------
Once we created the `dev` branch and verified that it exists, we can create some tables and add some data.

We create two tables under the `dev` branch:
- `salaries`
- `totals_stats`

These tables list the salaries per player per year and their stats per year.

To create the data we:

1. switch our branch context to dev
2. create the table
3. insert the data from an existing csv file. This csv file is already stored locally on the demo machine. A production use case would likely take feeds from official data sources

In [28]:
spark.sql("USE REFERENCE dev IN dev_catalog")

DataFrame[refType: string, name: string, hash: string]

In [29]:
spark.sql("""DROP TABLE dev_catalog.nba.salaries""")

DataFrame[]

In [30]:
# Creating `salaries` table
spark.sql("""CREATE TABLE IF NOT EXISTS dev_catalog.nba.salaries (Season STRING, Team STRING, Salary STRING, Player STRING) USING iceberg""")

DataFrame[]

In [31]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW salaries_table USING csv 
            OPTIONS (path "../datasets/nba/salaries.csv", header true)""")

DataFrame[]

In [32]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW salaries_table2 USING csv 
            OPTIONS (path "../datasets/nba/salaries.csv", header true)""")

DataFrame[]

In [33]:
spark.sql('INSERT INTO dev_catalog.nba.salaries SELECT * FROM salaries_table')

DataFrame[]

In [34]:
spark.sql("""DROP TABLE dev_catalog.nba.totals_stats""")

DataFrame[]

In [35]:
# Creating `totals_stats` table

In [36]:
spark.sql("""CREATE TABLE IF NOT EXISTS dev_catalog.nba.totals_stats (
             Season STRING, Age STRING, Team STRING, ORB STRING, DRB STRING, TRB STRING, AST STRING, STL STRING, 
             BLK STRING, TOV STRING, PTS STRING, Player STRING, RSorPO STRING) 
             USING iceberg""")

DataFrame[]

In [37]:
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW stats_table USING csv 
            OPTIONS (path "../datasets/nba/totals_stats.csv", header true)""")

DataFrame[]

In [38]:
spark.sql('SELECT * from stats_table')

DataFrame[Season: string, Age: string, Team: string, ORB: string, DRB: string, TRB: string, AST: string, STL: string, BLK: string, TOV: string, PTS: string, Player: string, RSorPO: string]

In [39]:
spark.sql('SELECT * from dev_catalog.nba.totals_stats')

DataFrame[Season: string, Age: string, Team: string, ORB: string, DRB: string, TRB: string, AST: string, STL: string, BLK: string, TOV: string, PTS: string, Player: string, RSorPO: string]

In [40]:
spark.sql('INSERT INTO dev_catalog.nba.totals_stats SELECT * FROM stats_table')

DataFrame[]

Now we count the rows in our tables to ensure they are the same number as the csv files. Note we use the `table@branch` notation which overrides the context set by a `USE REFERENCE` command.

In [41]:
table_count = spark.sql("select count(*) from dev_catalog.nba.`salaries@dev`").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from salaries_table").toPandas().values[0][0]
print(table_count)
print(csv_count)
assert table_count == csv_count
print(table_count)

table_count = spark.sql("select count(*) from dev_catalog.nba.`totals_stats@dev`").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from stats_table").toPandas().values[0][0]
print(table_count)
print(csv_count)
assert table_count == csv_count
print(table_count)

50
50
50
92
92
92


Check generated tables
----------------------------
Since we have been working solely on the `dev` branch, where we created 2 tables and added some data,
let's verify that the `main` branch was not altered by our changes.

In [42]:
spark.sql("USE REFERENCE main IN dev_catalog").toPandas()
try:
    spark.sql("SHOW TABLES IN dev_catalog").toPandas()
except:
    print("Error in 0.9.0 of Nessie causes an exception when no tables found")

And on the `dev` branch we expect to see two tables

In [43]:
spark.sql("USE REFERENCE dev IN dev_catalog").toPandas()
spark.sql("SHOW TABLES IN dev_catalog").toPandas()

,namespace,tableName
0,nba,salaries
1,nba,allstar_games_stats
2,nba,totals_stats


We can also verify that the `dev` and `main` branches point to different commits

In [44]:
spark.sql("LIST REFERENCES IN dev_catalog").toPandas()

,refType,name,hash
0,Branch,main,da0ea1e71d020987d2f2686deb6c9e80266b7d1e338903...
1,Branch,dev,cd00204ca3abf299b495bdcff84e907784948d76112595...


Dev promotion into main
-----------------------
Once we are done with our changes on the `dev` branch, we would like to merge those changes into `main`.
We merge `dev` into `main` via the Spark sql `merge` command.
Both branches should be at the same revision after merging/promotion.

In [45]:
spark.sql("MERGE BRANCH dev INTO main IN dev_catalog").toPandas()

,name,hash
0,main,cd00204ca3abf299b495bdcff84e907784948d76112595...


Check tables on main again
----------------------------
Since we have been working solely on the `dev` branch, where we created 2 tables and added some data,
let's verify that the `main` branch was not altered by our changes.

In [46]:
spark.sql("USE REFERENCE main IN dev_catalog").toPandas()
try:
    spark.sql("SHOW TABLES IN dev_catalog").toPandas()
except:
    print("Error in 0.9.0 of Nessie causes an exception when no tables found")

In [47]:
table_count = spark.sql("select count(*) from dev_catalog.nba.`salaries`").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from salaries_table").toPandas().values[0][0]
print(table_count)
print(csv_count)
assert table_count == csv_count
print(table_count)

table_count = spark.sql("select count(*) from dev_catalog.nba.`totals_stats`").toPandas().values[0][0]
csv_count = spark.sql("select count(*) from stats_table").toPandas().values[0][0]
print(table_count)
print(csv_count)
assert table_count == csv_count
print(table_count)

50
50
50
92
92
92


Cleanup branches
-----------------------

In [48]:
spark.sql("DROP BRANCH dev IN dev_catalog")



DataFrame[status: string]